# Demo Subcyle example
This notebook demonstartes the capabilities comparable to current cycle sheets without weather. It uses MultiStores to represent containers for MPs and TPs. 

In [1]:
import datetime, time
import simpy

import shapely.geometry
from simplekml import Kml, Style

import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot
import uuid

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}
keep_resources = {}

## Definition of Site

In [2]:
# The generic site class
Site = type(
    "Site",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.Locatable,  # Add coordinates to extract distance information and visualize
        core.HasMultiContainer,  # Add information on the material available at the site
        core.HasResource,
    ),  # Add information on serving equipment
    {},
)  # The dictionary is empty because the site type is generic

# Information on the extraction site - the "from site" - the "win locatie"
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)  # lon, lat

data_from_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Winlocatie",  # The name of the site
    "ID": str(uuid.uuid4()),  # For logging purposes
    "geometry": location_from_site,  # The coordinates of the project site
    "store_capacity": 4,
    "initials": [
        {"id": "MP", "level": 5, "capacity": 10},
        {"id": "TP", "level": 5, "capacity": 10},
    ],
}  # The actual volume of the site

# Information on the dumping site - the "to site" - the "dump locatie"
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)  # lon, lat

data_to_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Dumplocatie",  # The name of the site
    "ID": str(uuid.uuid4()),  # For logging purposes
    "geometry": location_to_site,  # The coordinates of the project site
    "store_capacity": 4,
    "initials": [
        {"id": "MP", "level": 0, "capacity": 5},
        {"id": "TP", "level": 0, "capacity": 5},
    ],
}  # The actual volume of the site (empty of course)

# The two objects used for the simulation
from_site = Site(**data_from_site)
to_site = Site(**data_to_site)

init
{'id': 'MP', 'level': 5, 'capacity': 10}
{'id': 'TP', 'level': 5, 'capacity': 10}
init
{'id': 'MP', 'level': 0, 'capacity': 5}
{'id': 'TP', 'level': 0, 'capacity': 5}


## Definition of Vessels

In [3]:
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.MultiContainerDependentMovable,  # A moving container, so capacity and location
        core.Processor,  # Allow for loading and unloading
        core.HasResource,  # Add information on serving equipment
        core.HasCosts,  # Add information on costs
        core.LoadingFunction,  # Add a loading function
        core.UnloadingFunction,  # Add an unloading function
        # SiteRegistry,
    ),
    {"key": "MultiStoreHopper"},
)

# print(SiteRegistry.inspect("MultiStoreHopper"))
# For more realistic simulation you might want to have speed dependent on the volume carried by the vessel
def compute_v_provider(v_empty, v_full):
    return lambda x: 10

# TSHD variables
data_hopper = {
    "env": my_env,  # The simpy environment
    "name": "Hopper 01",  # Name
    "ID": str(uuid.uuid4()),  # For logging purposes
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "store_capacity": 4,
    "initials": [
        {"id": "MP", "level": 0, "capacity": 2},
        {"id": "TP", "level": 0, "capacity": 2},
    ],  # Capacity of the hopper - "Beunvolume"
    "compute_v": compute_v_provider(5, 4.5),  # Variable speed
    "weekrate": 7,
}

hopper = TransportProcessingResource(**data_hopper)

init
{'id': 'MP', 'level': 0, 'capacity': 2}
{'id': 'TP', 'level': 0, 'capacity': 2}


## Definition of process
The process is introduced in three parts:
- Loading process
- Unloading process
- High level process

### Definition of Loading process
The loging process consists of simulation of a cyclesheet. There are some preparatory actions, followed by a loop to load MPs, then some changes on the crane configuration before the TPs are loaded in a loop again. 

In [4]:
# definition of loading
loading = []

basic_activity_data1= {"env"  : my_env,
                      "name" : "MP loading hook on rigging",
                      "registry": registry,
                      "ID":str(uuid.uuid4()),  # For logging purposes
                      "duration" : 45,
                      "additional_logs": [hopper],
                      "postpone_start": True,
                      }
loading.append(model.BasicActivity(**basic_activity_data1))

shift_amount_loading_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Transfer MP",  # We are moving soil
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "processor": hopper,
    "origin": from_site,
    "destination": hopper,
    "amount": 1,
    "duration": 120,
    "id_": "MP",
    "postpone_start": True,
}
activity_mp_loading = model.ShiftAmountActivity(**shift_amount_loading_data)
activity_mp_loading_seq_data = {
    "env": my_env,
    "name": "loading MP seq",
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "sub_processes": [activity_mp_loading],
    "postpone_start": True,
}
activity_mp_loading_seq = model.SequentialActivity(**activity_mp_loading_seq_data)
while_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "loading MP while",  # We are moving soil
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "sub_process": activity_mp_loading_seq,
    "condition_event": [{"or":[{"type":"container", "concept": hopper, "state":"full", "id_":"MP"},
                                {"type":"container", "concept": from_site, "state":"empty", "id_":"MP"}]
                        }],
    "postpone_start": True,
}
loading.append(model.WhileActivity(**while_data))

basic_activity_data2= {"env"  : my_env,
                      "name" : "MP loading hook off rigging",
                      "registry": registry,
                      "ID":str(uuid.uuid4()),  # For logging purposes
                      "duration" : 15,
                      "additional_logs": [hopper],
                      "postpone_start": True,
                      }
loading.append(model.BasicActivity(**basic_activity_data2))

basic_activity_data3= {"env"  : my_env,
                      "name" : "TP loading hook on rigging",
                      "registry": registry,
                      "ID":str(uuid.uuid4()),  # For logging purposes
                      "duration" : 30,
                      "additional_logs": [hopper],
                      "postpone_start": True,
                      }
loading.append(model.BasicActivity(**basic_activity_data3))

shift_amount_loading_data2 = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Transfer TP",  # We are moving soil
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "processor": hopper,
    "origin": from_site,
    "destination": hopper,
    "amount": 1,
    "duration": 10,
    "id_": "TP",
    "postpone_start": True,
}
activity_tp_loading = model.ShiftAmountActivity(**shift_amount_loading_data2)
activity_tp_loading_seq_data = {
    "env": my_env,
    "name": "loading MP seq",
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "sub_processes": [activity_tp_loading],
    "postpone_start": True,
}
activity_tp_loading_seq = model.SequentialActivity(**activity_tp_loading_seq_data)
while_data2 = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "loading MP while",  # We are moving soil
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "sub_process": activity_tp_loading_seq,
    "condition_event": [{"or":[{"type":"container", "concept": hopper, "state":"full", "id_":"TP"},
                                {"type":"container", "concept": from_site, "state":"empty", "id_":"TP"}]
                          }],
    #"condition_event": [{"type":"container", "concept": hopper, "state":"full", "id_":"TP"}],
    "postpone_start": True,
}
loading.append(model.WhileActivity(**while_data2))

basic_activity_data4= {"env"  : my_env,
                      "name" : "TP loading hook off rigging",
                      "registry": registry,
                      "ID":str(uuid.uuid4()),  # For logging purposes
                      "duration" : 15,
                      "additional_logs": [hopper],
                      "postpone_start": True,
                      }
loading.append(model.BasicActivity(**basic_activity_data4))

sequential_activity_data1 = {
    "env": my_env,
    "name": "loading",
    "ID": str(uuid.uuid4()),  # For logging purposes"registry": registry,
    "registry": registry,
    "sub_processes": loading,
    "postpone_start": True,
}

loading_activity = model.SequentialActivity(**sequential_activity_data1)


while Activity keep_resources []
while Activity keep_resources []


### Definition of Unloading
The unloading consists of unloading an MP and a TP with some auxilary actions.

In [5]:
# definition of unloading
#
unloading = []
move_activity_transit_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "sailing transit",  # We are moving soil
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "mover": hopper,
    "destination": to_site,
    "postpone_start": True,
}
unloading.append(model.MoveActivity(**move_activity_transit_data))
basic_activity_data20= {"env"  : my_env,
                      "name" : "MP preparing for installation",
                      "registry": registry,
                      "ID":str(uuid.uuid4()),  # For logging purposes
                      "duration" : 45,
                      "additional_logs": [hopper],
                      "postpone_start": True,
                      }
unloading.append(model.BasicActivity(**basic_activity_data20))
shift_amount_unloading_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Install MP",  # We are moving soil
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "processor": hopper,
    "origin": hopper,
    "destination": to_site,
    "amount": 1,
    "duration": 120,
    "id_": "MP",
    "postpone_start": True,
}
unloading.append(model.ShiftAmountActivity(**shift_amount_unloading_data))
basic_activity_data21= {"env"  : my_env,
                      "name" : "Installing MP",
                      "registry": registry,
                      "ID":str(uuid.uuid4()),  # For logging purposes
                      "duration" : 45,
                      "additional_logs": [hopper],
                      "postpone_start": True,
                      }
unloading.append(model.BasicActivity(**basic_activity_data21))
basic_activity_data22= {"env"  : my_env,
                      "name" : "Prepare TP installation",
                      "registry": registry,
                      "ID":str(uuid.uuid4()),  # For logging purposes
                      "duration" : 45,
                      "additional_logs": [hopper],
                      "postpone_start": True,
                      }
unloading.append(model.BasicActivity(**basic_activity_data22))

shift_amount_unloading_data2 = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Install TP",  # We are moving soil
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "processor": hopper,
    "origin": hopper,
    "destination": to_site,
    "amount": 1,
    "duration": 120,
    "id_": "TP",
    "postpone_start": True,
}
unloading.append(model.ShiftAmountActivity(**shift_amount_unloading_data2))

basic_activity_data23= {"env"  : my_env,
                      "name" : "TP finalize installation",
                      "registry": registry,
                      "ID":str(uuid.uuid4()),  # For logging purposes
                      "duration" : 45,
                      "additional_logs": [hopper],
                      "postpone_start": True,
                      }
unloading.append(model.BasicActivity(**basic_activity_data23))

sequential_activity_data2 = {
    "env": my_env,
    "name": "unloading",
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "sub_processes": unloading,
    "postpone_start": True,
}
sequential_activity = model.SequentialActivity(**sequential_activity_data2)

while_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "unloading while",  # We are moving soil
    "ID":str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "sub_process": sequential_activity,
    "condition_event": [{"or":[{"type":"container", "concept": to_site, "state":"full", "id_":"TP"},
                                {"type":"container", "concept": hopper, "state":"empty", "id_":"TP"}]
                          }],
    #"condition_event": [{"type":"container", "concept": hopper, "state":"empty", "id_":"TP"}],
    "postpone_start": True,
}
unloading_activity = model.WhileActivity(**while_data)

while Activity keep_resources []


### Definition of High level Process
The high level process is based on the following steps:
- While the destination is not full do:
    - Sail empty
    - Loading
    - Sail full
    - Unloading

In [6]:
# definition of main cycle

single_run = []

move_activity_to_harbor_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "sailing empty",  # We are moving soil
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "mover": hopper,
    "destination": from_site,
    "postpone_start": True,
}
single_run.append(model.MoveActivity(**move_activity_to_harbor_data))

single_run.append(loading_activity)

move_activity_to_site_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "sailing filled",  # We are moving soil
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "mover": hopper,
    "destination": to_site,
    "postpone_start": True,
}
single_run.append(model.MoveActivity(**move_activity_to_site_data))

single_run.append(unloading_activity)

sequential_activity_data3 = {
    "env": my_env,
    "name": "Single run process",
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "sub_processes": single_run,
    "postpone_start": True,
}
activity = model.SequentialActivity(**sequential_activity_data3)

while_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "single run while",  # We are moving soil
    "ID": str(uuid.uuid4()),  # For logging purposes
    "registry": registry,
    "sub_process": activity,
    "condition_event": [{"type":"container", "concept": to_site, "state":"full", "id_":"TP"}],
    "postpone_start": False,
}
while_activity = model.WhileActivity(**while_data)


while Activity keep_resources []
get_full_event : TP
start get_available
start event instance None


In [8]:
my_env.run()


et_callback
{'TP': {2: <Event() object at 0x2067f789688>}}
origin store after get: [{'id': 'TP', 'level': 1, 'capacity': 2}, {'id': 'MP', 'level': 0, 'capacity': 2}]
after get
after check_down time
destination store before put: [{'id': 'MP', 'level': 3, 'capacity': 5}, {'id': 'TP', 'level': 3, 'capacity': 5}]
<FilterStoreGet() object at 0x2067f79a4c8>
put_callback - id_ MP
{'TP': {5: <Event() object at 0x2067f1a4908>}}
destination store after put: [{'id': 'TP', 'level': 3, 'capacity': 5}, {'id': 'MP', 'level': 4, 'capacity': 5}]
after put
processor end process
Processed 1 of MP:
  by:          Hopper 01
  origin        Hopper 01  contains: 0 of MP
  destination:  Dumplocatie contains: 4 of MP
after shift amount : {<simpy.resources.resource.Resource object at 0x000002067F78AD88>: <Request() object at 0x2067f789ec8>, <simpy.resources.resource.Resource object at 0x000002067F78A808>: <Request() object at 0x2067f79a808>}
keep resources []
release destination : {<simpy.resources.resource.Res

In [9]:
log_df = pd.DataFrame(hopper.log)
data = log_df[["Message", "ActivityState", "Timestamp", "Value", "ActivityID"]]
data = data.drop_duplicates()
data = data[data['ActivityState']=='START']
data

,Message,ActivityState,Timestamp,Value,ActivityID
0,sailing empty,START,1970-01-01 01:00:00.000000,0,846e72e3-8e64-4720-977d-0df00b0cbcaa
2,MP loading hook on rigging,START,1970-01-01 01:00:00.000000,45,2e95f671-0a9e-4a65-a3aa-e3c09cbc92be
4,transfer MP to Hopper 01,START,1970-01-01 01:00:45.000000,1,f9881f28-2a6d-48eb-8ead-f6b165bf65a7
8,transfer MP to Hopper 01,START,1970-01-01 01:02:45.000000,1,f9881f28-2a6d-48eb-8ead-f6b165bf65a7
12,MP loading hook off rigging,START,1970-01-01 01:04:45.000000,15,48bea753-5d03-447f-b7fc-38cada0b038f
...,...,...,...,...,...
152,transfer MP to Dumplocatie,START,1970-01-01 03:03:24.122956,1,fcb736cf-b3f4-4a5f-8b99-f8dfec657fe2
156,Installing MP,START,1970-01-01 03:05:24.122956,45,98632eff-bf58-4536-ab56-2767815f8e47
158,Prepare TP installation,START,1970-01-01 03:06:09.122956,45,b350b682-9e47-4a08-a33a-1edd754adb45
160,transfer TP to Dumplocatie,START,1970-01-01 03:06:54.122956,1,c9d243b0-b0b8-4c8f-a2ea-6f999bda0029


In [13]:
# Loading first and second run
data.iloc[:10]

,Message,ActivityState,Timestamp,Value,ActivityID
0,sailing empty,START,1970-01-01 01:00:00,0,846e72e3-8e64-4720-977d-0df00b0cbcaa
2,MP loading hook on rigging,START,1970-01-01 01:00:00,45,2e95f671-0a9e-4a65-a3aa-e3c09cbc92be
4,transfer MP to Hopper 01,START,1970-01-01 01:00:45,1,f9881f28-2a6d-48eb-8ead-f6b165bf65a7
8,transfer MP to Hopper 01,START,1970-01-01 01:02:45,1,f9881f28-2a6d-48eb-8ead-f6b165bf65a7
12,MP loading hook off rigging,START,1970-01-01 01:04:45,15,48bea753-5d03-447f-b7fc-38cada0b038f
14,TP loading hook on rigging,START,1970-01-01 01:05:00,30,d865cc8f-3e5c-46b7-856d-1aaea200b203
16,transfer TP to Hopper 01,START,1970-01-01 01:05:30,1,b14807e0-0fcd-4639-b928-9d47e8139d8e
20,transfer TP to Hopper 01,START,1970-01-01 01:05:40,1,b14807e0-0fcd-4639-b928-9d47e8139d8e
24,TP loading hook off rigging,START,1970-01-01 01:05:50,15,91dfa4b1-84b9-4847-b49f-22b24b9fb602
26,sailing filled,START,1970-01-01 01:06:05,0,3b6c76c6-4a26-4a0b-b746-b14c2dddf325


In [16]:
# Loading third run
data.iloc[48:]

,Message,ActivityState,Timestamp,Value,ActivityID
128,sailing empty,START,1970-01-01 02:27:18.473774,0,846e72e3-8e64-4720-977d-0df00b0cbcaa
130,MP loading hook on rigging,START,1970-01-01 02:43:01.298365,45,2e95f671-0a9e-4a65-a3aa-e3c09cbc92be
132,transfer MP to Hopper 01,START,1970-01-01 02:43:46.298365,1,f9881f28-2a6d-48eb-8ead-f6b165bf65a7
136,MP loading hook off rigging,START,1970-01-01 02:45:46.298365,15,48bea753-5d03-447f-b7fc-38cada0b038f
138,TP loading hook on rigging,START,1970-01-01 02:46:01.298365,30,d865cc8f-3e5c-46b7-856d-1aaea200b203
140,transfer TP to Hopper 01,START,1970-01-01 02:46:31.298365,1,b14807e0-0fcd-4639-b928-9d47e8139d8e
144,TP loading hook off rigging,START,1970-01-01 02:46:41.298365,15,91dfa4b1-84b9-4847-b49f-22b24b9fb602
146,sailing filled,START,1970-01-01 02:46:56.298365,0,3b6c76c6-4a26-4a0b-b746-b14c2dddf325
148,sailing filled,START,1970-01-01 03:02:39.122956,0,e5674d5f-d7e4-43d6-a627-10f12a6d488f
150,MP preparing for installation,START,1970-01-01 03:02:39.122956,45,07f17555-6ae2-466e-a1e3-a4d858b5caa2


In [19]:
# unloading first run
data.iloc[10:25]

,Message,ActivityState,Timestamp,Value,ActivityID
28,sailing filled,START,1970-01-01 01:21:47.824591,0,e5674d5f-d7e4-43d6-a627-10f12a6d488f
30,MP preparing for installation,START,1970-01-01 01:21:47.824591,45,07f17555-6ae2-466e-a1e3-a4d858b5caa2
32,transfer MP to Dumplocatie,START,1970-01-01 01:22:32.824591,1,fcb736cf-b3f4-4a5f-8b99-f8dfec657fe2
36,Installing MP,START,1970-01-01 01:24:32.824591,45,98632eff-bf58-4536-ab56-2767815f8e47
38,Prepare TP installation,START,1970-01-01 01:25:17.824591,45,b350b682-9e47-4a08-a33a-1edd754adb45
40,transfer TP to Dumplocatie,START,1970-01-01 01:26:02.824591,1,c9d243b0-b0b8-4c8f-a2ea-6f999bda0029
44,TP finalize installation,START,1970-01-01 01:28:02.824591,45,369fba05-02a7-4d9c-a73b-925df6cc259d
46,sailing filled,START,1970-01-01 01:28:47.824591,0,e5674d5f-d7e4-43d6-a627-10f12a6d488f
48,MP preparing for installation,START,1970-01-01 01:28:47.824591,45,07f17555-6ae2-466e-a1e3-a4d858b5caa2
50,transfer MP to Dumplocatie,START,1970-01-01 01:29:32.824591,1,fcb736cf-b3f4-4a5f-8b99-f8dfec657fe2


Log of the high level process

In [11]:
while_log_df = pd.DataFrame(while_activity.log)
while_data = while_log_df[["Message", "ActivityState", "Timestamp", "Value", "ActivityID"]]
while_data = while_data.drop_duplicates()
#while_data = while_data[data['ActivityState']=='START']
while_data

,Message,ActivityState,Timestamp,Value,ActivityID
0,conditional process single run while,START,1970-01-01 01:00:00.000000,-1,c556d006-18cb-4fcf-bc08-904ee4eb88eb
1,sub process Single run process,START,1970-01-01 01:00:00.000000,-1,c556d006-18cb-4fcf-bc08-904ee4eb88eb
2,sequential Single run process,START,1970-01-01 01:00:00.000000,-1,c556d006-18cb-4fcf-bc08-904ee4eb88eb
3,sub process sailing empty,START,1970-01-01 01:00:00.000000,-1,c556d006-18cb-4fcf-bc08-904ee4eb88eb
4,sub process sailing empty,STOP,1970-01-01 01:00:00.000000,-1,c556d006-18cb-4fcf-bc08-904ee4eb88eb
5,sub process loading,START,1970-01-01 01:00:00.000000,-1,c556d006-18cb-4fcf-bc08-904ee4eb88eb
6,sub process loading,STOP,1970-01-01 01:06:05.000000,-1,c556d006-18cb-4fcf-bc08-904ee4eb88eb
7,sub process sailing filled,START,1970-01-01 01:06:05.000000,-1,c556d006-18cb-4fcf-bc08-904ee4eb88eb
8,sub process sailing filled,STOP,1970-01-01 01:21:47.824591,-1,c556d006-18cb-4fcf-bc08-904ee4eb88eb
9,sub process unloading while,START,1970-01-01 01:21:47.824591,-1,c556d006-18cb-4fcf-bc08-904ee4eb88eb
